## Next.js authentication with Auth.js

###### Auth.js installation and setting up.

First we have to install in our project (please check official documentation, because this solution may be outdated)

In [ ]:
npm install next-auth@beta

Then we have to create an .env.local file with environment variables. We can do it automatically by entering command:

In [ ]:
npx auth secret

Now we should have a new file in our project ".env.local" with the code 

In [ ]:
AUTH_SECRET="AS+BzGp3JFhapgWirKUEfltWsTG9tc5GDuv/pplwHxI=" # Added by `npx auth`. Read more: https://cli.authjs.dev

We have to add an AUTH_URL variable.

In [ ]:
AUTH_SECRET="AS+BzGp3JFhapgWirKUEfltWsTG9tc5GDuv/pplwHxI=" # Added by `npx auth`. Read more: https://cli.authjs.dev
AUTH_URL = http://localhost:3000/api/auth

Now, let's create a configuration file auth.ts in a root folder (if you want, you can create this file in any folder). Copy the code from the official Auth.js documentation and add GitHub to providers array. (it can be google provider, facebook, etc.)

In [ ]:
import NextAuth from "next-auth"
import GitHub from "next-auth/providers/github"
 
export const { handlers: {GET, POST} , signIn, signOut, auth } = NextAuth({ 
    ## {GET, POST} is important for export in route in folder app/api/[...nextauth] (will be created in next steps)
  providers: [ GitHub],  # Add GitHub to providers array
})

In our project we are using GitHub provider, so we have to create a new GitHub app. To do this, visit the GitHub page, sign in, and click on your avatar on the top right. 
In the menu click settings, and on the left navbar click Developer Settings (at the very bottom). 
Then click OAuthApps and register new application.

Now fill the input fields.

Application name - Next14 tutorial (the name is up to you).<br>
Homepage URL - http://localhost:3000<br>
Authorization callback URL - http://localhost:3000/api/auth/callback/github (or google if we use google, etc).

New page with client id will show up. Copy client id and generate new secret. In the .env.local file add:

In [ ]:
AUTH_GITHUB_ID="Oi23liF2fagexcPGdde8"
AUTH_GITHUB_SECRET="fc89647a01e9ce53ab94bb1b2c31acaee959595f"

We can use them in the configuration file.

In [ ]:
import NextAuth from "next-auth"
import GitHub from "next-auth/providers/github"
 
export const { handlers, signIn, signOut, auth } = NextAuth({
  providers: [ GitHub],  # Add GitHub to providers array
})

### OR ###

import NextAuth from "next-auth"
import GitHub from "next-auth/providers/github"
 
export const { handlers, signIn, signOut, auth } = NextAuth({
  providers: [
    GitHub({
      clientId: process.env.AUTH_GITHUB_ID, ## refer to variable names in .env.local
      clientSecret: process.env.AUTH_GITHUB_SECRET, ## refer to variable names in .env.local
    })
  ],
})

Now we can use function from NextAuth() in our app. Let's write a function and use it in our Login page.

In [ ]:
## page.tsx (login page)

import { signIn } from '@/lib/auth';

const LoginPage = () => {

  const handleGithubLogin = async () => {  ## this function will redirect us to authentication GitHub authentication page
    "use server";
    await signIn("github")  ## we use github provider, but ther is a list of many which we can also use.
  }

  return (
    <div>
      <form action={handleGithubLogin}>
        <button>Login with Github</button> 
      </form>
    </div>
  )
}

export default LoginPage

Next step is to create an API endpoint, so in folder app/api create a folder [...nextauth] (otherwise you would have to create all required endpoints (for login, error, logout, etc.).

Create a file route.ts inside that folder.

This route will handle all the POST and GET methods.

In [ ]:
import { handlers } from "@/lib/auth"
export const { GET, POST } = handlers

Now we can login and for exampe conditionally render button (when the user is logged in, button text will say "logout", and so on).

In [ ]:
## Navbar.tsx (Next.js tutorial)
import Link from "next/link"
import Links from "./links/Links"
import styles from './navbar.module.css'
import { auth } from "@/lib/auth"  ## importing auth from lib folder

const Navbar = async () => {

  const session = await auth(); # this is a server side component so we can declare variable and get returned value from auth()

  return (
    <div className="flex flex-row justify-between h-24 items-center text-white">
      <Link href={"/"} className="text-lg font-bold">Logo</Link>
      <div>
        <Links session={session} />  # we pass the session as a prop to Links component.
      </div>
    </div>
  )
}

export default Navbar

Now we can use the session in our child component.

###### Important:

Why we can't use session in our child component? Because we declared it as a client component (we have useState hook in this component). That means that we can't decalare Links component as an async function.

In [ ]:
'use client'

import React, { useState } from "react";
import Link from "next/link";
import NavLink from "./navLink/NavLink";
import { LinkObject } from "@/lib/types/types";
import Image from "next/image";
import { handleGithubLogout } from "@/lib/actions";
import { Session } from "next-auth";

const links = [
  {
    title: "Homepage",
    path: "/",
  },
  {
    title: "About",
    path: "/about",
  },
  {
    title: "Contact",
    path: "/contact",
  },
  {
    title: "Blog",
    path: "/blog",
  },
];

# we have to add additional property to the interface Session from next-auth. Otherwise session.user?.isAdmin will show TS error
interface Session {
  user : {
    isAdmin?: boolean
  }
}

const Links = ({session}: {session: Session}) => {

  # TEMPORARY
  const isAdmin = true;

  const [open, setOpen] = useState(false);
  console.log(session)

  return (
    <div>
      <div className="max-sm:hidden md:flex items-center gap-2.5" >
        {links.map((link) => {
          return (
            <NavLink key={link.title + link.path} item={link} />
          )
        })
      }
        {session?.user ? (
          <>
              ## no TS error below because we have added new property to Session interface
              {session.user?.isAdmin && <NavLink item={{title: "Admin", path: "/admin"}} />}
            <form action={handleGithubLogout}>
              <button 
                className="p-2.5 cursor-pointer font-bold bg-white text-slate-900 rounded-sm"
                >
                Logout
              </button>
            </form>
          </>
          ) : (
            <NavLink item={{title: "Login", path: "/login"}} />
          )
        }
      </div>
    </div>
  )
}

export default Links

##### Adding user to DB

First we have to check if the use exists in our database. If not we have to create a new user and add it to DB.
We have to modify auth.js file (lib directory).

In [ ]:
## auth.ts

import NextAuth from "next-auth"
import GitHub from "next-auth/providers/github"
import { connectToDb } from "./utils"
import { User } from "./models"
 
export const { handlers, signIn, signOut, auth } = NextAuth({
  providers: [
    GitHub({
      clientId: process.env.AUTH_GITHUB_ID,
      clientSecret: process.env.AUTH_GITHUB_SECRET,
    })
  ],
    
  # we have to add now callbacks
  callbacks: {
    async signIn({user, account, profile}) { # user, account, profile are objects with data
      console.log(profile)
      if (account?.provider === "github") {
        connectToDb();
        try {
          const user = await User.findOne({email: profile?.email}); # look for the user with the same email in DB
            
          # if the user is not found, it means it doesn't exist in the DB, so we can create a new one and add it to DB
          if (!user) {
            ## create new user according to userSchema in models.js
            const newUser = new User({
              username: profile?.login, # in github there is login, in other may be for example name, or username, 
              email: profile?.email,
              img: profile?.avatar_url,
            });

            await newUser.save();
          }

        } catch (error) {
          console.log(error)
          return false # if there is an error - return false.
        }
      }
      return true # if there are no errors, return true
    },
  }
})

###### Important:

We may have an error because we set in our userSchema that the password is require. We have to remove this property from the schema.

In [ ]:
const userSchema = new mongoose.Schema(
    {
        username: {
            type: String,
            required: true,
            min: 3,
            max: 20
        },
        email: {
            type: String,
            required: true,
            unique: true,
            max: 50
        },
        password: {
            type: String,
#             required: true # <= comment or remove this line
        },
        #...
            